# Extract Annotated User Sentences from Taskmaster-2
Extract annotated user sentences for training an entity recognition model

## Get training data
The goal of the chatbot model is to interpret the user inputs, and identify intents. 
I think the flow is something like, identify user intents > generate response/follow-up question > interpret user intent again > repeat. > end conversation.

So, the steps for creating a chatbot:  
1. Model capable of recognizing entities  
2. Model capable of recognizing intents (greeting, question, ending)  
3. Put it together

### Pull out the User inputs that contain annotations from the json
json traversal of the utterances column, i don't need the ids

spacy requires training data as a list of tuples - more specifically, each tuple contains a string and a dictionary.  
The dictionary has one key "entities" which has a list for a value. the value contains a tuple that holds the start_index, end_index and annotation.

In [2]:
import json
data = json.load(open("c:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\data\\restaurant-search.json"))

In [3]:
from tqdm import tqdm
# dump_file = 'c:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\data\\user-annotated.json'
conversations = []

for  conversation in tqdm(data):
    # temp = []
    for sentence in conversation['utterances']:
        if len(sentence) == 4:
            conversations.append(sentence)

100%|██████████| 3276/3276 [00:00<00:00, 272748.82it/s]


In [4]:
print(conversations[0])

{'index': 1, 'speaker': 'USER', 'text': 'Hi, I need help finding a sandwich shop to go eat at.', 'segments': [{'start_index': 26, 'end_index': 34, 'text': 'sandwich', 'annotations': [{'name': 'restaurant.type.food'}]}]}


To get the properly formatted list, i need to first get a list of strings (i.e. the sentences).  
As well as a list of dicts

In [18]:
import ast

user_sentence = []
temp = []
user_entities = []

for sentence in conversations:
    user_sentence.append(sentence['text'])
    temp.append('{{"entities": [({}, {}, "{}")]}}'.format(sentence['segments'][0]['start_index'], sentence['segments'][0]['end_index'], sentence['segments'][0]['annotations'][0]['name']))

for ent in temp:
    user_entities.append(ast.literal_eval(ent))

# conversations[0]['segments'][0].keys()

In [19]:
type(user_ent[0]['entities'][0])

tuple

In [29]:
user_data = zip(user_sentence, user_entities)

In [30]:
user_data = tuple(user_data)

In [35]:
user_data[0][1]['entities']

[(26, 34, 'restaurant.type.food')]

In [36]:
user_data[0]

('Hi, I need help finding a sandwich shop to go eat at.',
 {'entities': [(26, 34, 'restaurant.type.food')]})

In [37]:
dump_file = "C:\\Users\\Ben\\Documents\\ebacs\\practical_language_processing\\ebacs_plp_project\\data\\user_training_data.json"

with open(dump_file, 'w', encoding='utf-8') as f:
    json.dump(user_data, f)